# Erick Iván Castro Granados

# Cálculo de VaR 

# ITAM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import randn, seed
from scipy.optimize import minimize
from scipy.stats import norm

In [34]:
### Datos iniciales 

monto = 10000000
nivelc1 = .95
nivelc2 = .99
cuantil1 = norm.ppf(nivelc1)
cuantil2 = norm.ppf(nivelc2)

In [6]:
port_path = "PreciosPortf.xlsx"
data = pd.read_excel(port_path, index_col=0)

#base
rend = data.pct_change().dropna()

#rendimiento anualizado
rend_ports = rend.mean() * 252
rend_ports

ASURB MF Equity       0.123539
GFNORTEO MF Equity    0.078811
PE&OLES* MF Equity    0.111053
WALMEX* MF Equity     0.082426
IENOVA* MF Equity     0.078997
dtype: float64

In [89]:
## Matriz varianzas- covarianzas
var_cov = rend.cov()
var_cov

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
ASURB MF Equity,0.000224,0.000086,0.000061,0.000036,0.000044
GFNORTEO MF Equity,0.000086,0.000341,0.000094,0.000082,0.000082
PE&OLES* MF Equity,0.000061,0.000094,0.000548,0.000041,0.000045
WALMEX* MF Equity,0.000036,0.000082,0.000041,0.000215,0.000043
IENOVA* MF Equity,0.000044,0.000082,0.000045,0.000043,0.000226


In [10]:
def port_rend(weights, r):
    E_p = (r.mean() @ weights) * 252
    return E_p

def port_vol(weights, r):
    S_p = np.sqrt(weights.T @ (r.cov() * 252) @ weights)
    return S_p

In [11]:
N, M = rend.shape
w0  = np.random.randn(M)

def sum_weights(weights):
    return weights.sum() - 1 # == 0

def rend_esperado(w, E):
    return port_rend(w, rend) - E # == 0

e0 = 0.1
constraints = [
    {"type": "eq", "fun": sum_weights},
    {"type": "eq", "fun": lambda w: rend_esperado(w, e0)}
]

port1 = minimize(port_vol, w0, constraints=constraints, args=rend)
port1

     fun: 0.15715162621795464
     jac: array([0.17775704, 0.13851267, 0.16664604, 0.14176532, 0.13885481])
 message: 'Optimization terminated successfully.'
    nfev: 70
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.37295029, 0.00650116, 0.10717024, 0.2790069 , 0.23437141])

In [12]:
wp1 = port1.x
port_rend(wp1, rend), port_vol(wp1, rend)

(0.09999999999750425, 0.15715162621795464)

In [13]:
### Sacamos los pesos que minimicen el coeficiente de Sharpe
rf = 0.06

def min_func_sharpe(weights):
    return - (port_rend(weights, rend) - rf) / port_vol(weights, rend)

constraints = [
    {"type": "eq", "fun": sum_weights}
]

f_sharpe = minimize(min_func_sharpe, w0, constraints=constraints)

In [14]:
### Pesos usados para calcular el VaR
w_sharpe = f_sharpe.x
w_sharpe

array([ 0.73086632, -0.13937858,  0.17629872,  0.16409262,  0.06812092])

In [33]:
### Calculamos la varianza y desv estandar del portafolio
varp_diaria = w_sharpe @ (var_cov @ w_sharpe)
desvp_diaria = np.sqrt(varp_diaria)
desvp_anual = desvp_diaria * np.sqrt(252)
varp_anual = desvp_anual ** 2

In [38]:
### Calculamos el VaR 
VaR_95_anual = monto * cuantil1 * desvp_anual
VaR_99_anual = monto * cuantil2 * desvp_anual
VaR_95_diario = monto * cuantil1 * desvp_diaria
VaR_99_diario = monto * cuantil2 * desvp_diaria

In [98]:
## Calculamos el efecto de la diversificación 
vector_pos = w_sharpe * monto
vold_acc = np.sqrt(np.diagonal(var_cov))
vola_acc = vold_acc * np.sqrt(252)

In [103]:
### Suma del VaR diario y anual de cada una de las acciones

VaR_indiv_d_95 = np.sum(vector_pos * vold_acc * cuantil1)
VaR_indiv_a_95 = np.sum(vector_pos * vola_acc * cuantil1)

VaR_indiv_d_99 = np.sum(vector_pos * vold_acc * cuantil2)
VaR_indiv_a_99 = np.sum(vector_pos * vola_acc * cuantil2)

In [104]:
efecto_a_95 = VaR_indiv_a_95 - VaR_95_anual
efecto_a_99 = VaR_indiv_a_99 - VaR_99_anual

efecto_d_95 = VaR_indiv_d_95 - VaR_95_diario
efecto_d_99 = VaR_indiv_d_99 - VaR_99_diario

# Resultados:

In [108]:
print(f"El valor del VaR anual al 95% de confianza es: $ {VaR_95_anual:.2f}")
print(f"El valor del VaR anual al 99% de confianza es: $ {VaR_99_anual:.2f}")

print(f"El valor del VaR diario al 95% de confianza es: $ {VaR_95_diario:.2f}")
print(f"El valor del VaR diario al 99% de confianza es: $ {VaR_99_diario:.2f}")

El valor del VaR anual al 95% de confianza es: $ 3257080.84
El valor del VaR anual al 99% de confianza es: $ 4606551.59
El valor del VaR diario al 95% de confianza es: $ 205176.81
El valor del VaR diario al 99% de confianza es: $ 290185.47


In [109]:
print(f"El efecto de la diversificación anual al 95% de confianza es: $ {efecto_a_95:.2f}")
print(f"El efecto de la diversificación anual al 99% de confianza es: $ {efecto_a_99:.2f}")
print(f"El efecto de la diversificación diario al 95% de confianza es: $ {efecto_d_95:.2f}")
print(f"El efecto de la diversificación diario al 99% de confianza es: $ {efecto_d_99:.2f}")

El efecto de la diversificación anual al 95% de confianza es: $ 898484.58
El efecto de la diversificación anual al 99% de confianza es: $ 1270743.89
El efecto de la diversificación diario al 95% de confianza es: $ 56599.21
El efecto de la diversificación diario al 99% de confianza es: $ 80049.34
